In [5]:
import os
import json
import pandas as pd
from decimal import Decimal, getcontext
import collections

# 设置Decimal的精度
getcontext().prec = 50

def analyze_liquidation_data(input_dir, output_dir):
    """
    分析 input_dir 中的清算JSON文件，并将统计结果保存到 output_dir。
    
    假设:
    1. `priceInEth` 字段实际上是带有8位小数的 USD 价格。
    2. '抵押物' 是 currentATokenBalance > 0 且 usageAsCollateralEnabledOnUser == True 的资产。
    3. '贷款' 是 currentTotalDebt > 0 的资产。
    """
    
    # --- 1. 设置 ---
    if not os.path.isdir(input_dir):
        print(f"错误: 找不到输入目录: {input_dir}")
        print(f"请确保您的 .ipynb 文件保存在: '.../R1' 目录下")
        print(f"并且数据目录存在于: '.../R1/data/pool_data'")
        return

    # 创建输出目录，如果已存在则不报错
    os.makedirs(output_dir, exist_ok=True)
    
    # 使用 defaultdict 简化聚合逻辑
    collateral_stats = collections.defaultdict(lambda: {'total_value_usd': Decimal(0), 'count': 0, 'liquidation_threshold': '0'})
    debt_stats = collections.defaultdict(lambda: {'total_value_usd': Decimal(0), 'count': 0})
    
    all_datetimes = []
    
    # 价格假设：USD价格，8位小数
    PRICE_DECIMALS = Decimal(10**8)
    
    json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    
    if not json_files:
        print(f"错误: 在 {input_dir} 中未找到 .json 文件")
        return
        
    print(f"找到 {len(json_files)} 个 .json 文件。开始处理...")

    # --- 2. 处理每个文件 ---
    for file_name in json_files:
        file_path = os.path.join(input_dir, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # ######################################
            #  *** 这是修正的地方 ***
            #  我们收集数字时间戳 (timestamp)，而不是字符串日期 (datetime)
            # ######################################
            if 'liquidation_timestamp' in data:
                all_datetimes.append(data['liquidation_timestamp'])
            
            if 'pre_liquidation_snapshot' not in data:
                continue

            # --- 3. 处理快照中的每项资产 ---
            for asset in data['pre_liquidation_snapshot']:
                try:
                    symbol = asset['reserve']['symbol']
                    asset_decimals = Decimal(10**int(asset['reserve']['decimals']))
                    
                    # 假设 priceInEth 是带8位小数的USD价格
                    price_in_usd = Decimal(asset['reserve']['price']['priceInEth']) / PRICE_DECIMALS
                    
                    lt = asset['reserve']['reserveLiquidationThreshold']
                    
                    collateral_balance = Decimal(asset['currentATokenBalance'])
                    debt_balance = Decimal(asset['currentTotalDebt'])

                    # 检查抵押物
                    # 必须是：余额 > 0 且 用作抵押物=True
                    if asset['usageAsCollateralEnabledOnUser'] and collateral_balance > 0:
                        amount_human = collateral_balance / asset_decimals
                        value_usd = amount_human * price_in_usd
                        
                        collateral_stats[symbol]['total_value_usd'] += value_usd
                        collateral_stats[symbol]['count'] += 1
                        # 存储清算阈值 (假设同币种该值不变)
                        collateral_stats[symbol]['liquidation_threshold'] = lt

                    # 检查贷款
                    # 必须是：贷款余额 > 0
                    if debt_balance > 0:
                        amount_human = debt_balance / asset_decimals
                        value_usd = amount_human * price_in_usd
                        
                        debt_stats[symbol]['total_value_usd'] += value_usd
                        debt_stats[symbol]['count'] += 1
                        
                except (KeyError, TypeError, ValueError) as e:
                    # 捕获数据残缺或格式错误
                    print(f"警告: 跳过 {file_name} 中的一项资产，数据问题: {e}")
        
        except json.JSONDecodeError:
            print(f"警告: 跳过文件 {file_name}，非法的JSON格式。")
        except Exception as e:
            print(f"处理文件 {file_name} 时发生错误: {e}")

    # --- 4. 查找日期范围 ---
    if all_datetimes:
        min_datetime_ts = min(all_datetimes)
        max_datetime_ts = max(all_datetimes)
        
        # ######################################
        #  *** 这里的代码现在可以正常工作了 ***
        #  因为 min_datetime_ts 现在是一个数字 (例如 1760716721)
        #  而不是一个字符串 (例如 "2025-10-17 15:58:41")
        # ######################################
        min_datetime_str = pd.to_datetime(min_datetime_ts, unit='s')
        max_datetime_str = pd.to_datetime(max_datetime_ts, unit='s')
        print(f"\n已处理清算事件的时间范围 (UTC):")
        print(f"  最早: {min_datetime_str} (Timestamp: {min_datetime_ts})")
        print(f"  最晚: {max_datetime_str} (Timestamp: {max_datetime_ts})")
    else:
        print("\n未找到 'liquidation_timestamp' 字段。")

    # --- 5. 序列化并保存结果 ---
    
    # JSON 序列化 Decimal 对象的辅助类
    class DecimalEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, Decimal):
                return str(obj)  # 将 Decimal 转换为字符串
            return super(DecimalEncoder, self).default(obj)

    # 将 defaultdict 转换为普通 dict 以便JSON输出
    collateral_stats_dict = {k: v for k, v in collateral_stats.items()}
    debt_stats_dict = {k: v for k, v in debt_stats.items()}

    # 保存抵押物统计
    collateral_output_path = os.path.join(output_dir, "collateral_statistics.json")
    with open(collateral_output_path, 'w', encoding='utf-8') as f:
        json.dump(collateral_stats_dict, f, indent=4, cls=DecimalEncoder, ensure_ascii=False)
    
    print(f"\n成功保存抵押物统计数据到: {collateral_output_path}")

    # 保存贷款统计
    debt_output_path = os.path.join(output_dir, "debt_statistics.json")
    with open(debt_output_path, 'w', encoding='utf-8') as f:
        json.dump(debt_stats_dict, f, indent=4, cls=DecimalEncoder, ensure_ascii=False)
        
    print(f"成功保存贷款统计数据到: {debt_output_path}")

# --- 执行 ---
# 您的 .ipynb 文件位于 '.../R1' 目录
# 'data/pool_data' 是相对于 .ipynb 文件的输入路径
INPUT_DIR = 'data/pool_data'
# 'data/statistic' 是相对于 .ipynb 文件的输出路径
OUTPUT_DIR = 'data/statistic'

analyze_liquidation_data(INPUT_DIR, OUTPUT_DIR)

找到 42 个 .json 文件。开始处理...

已处理清算事件的时间范围 (UTC):
  最早: 2025-10-17 08:53:55 (Timestamp: 1760691235)
  最晚: 2025-10-30 13:37:13 (Timestamp: 1761831433)

成功保存抵押物统计数据到: data/statistic\collateral_statistics.json
成功保存贷款统计数据到: data/statistic\debt_statistics.json


我真的会疯

直接调用subgraph试一下